# Your task is to write a Python script that calculates both the crude death rate and the age-standardized death rate for all ages in both the United States and Uganda for 2019.

# Your answers should represent the whole population of each country, and the value should be given as deaths per 100,000 people, rounded to one decimal place.

### I imported the three datasets, one for the Age-specific death rates for COPD, another for the populations by age group, and another for the age distribution as defined by WHO. I then cleaned/filtered them to take only the necessary columns and be able to join the dataframes on 'Age group'. For the age group population data, I also created a Total value for each country and then melted the dataframe to convert the wide age group data into the 'Age group' and 'population' columns. For the calculations, I merged all three dataframes on 'Age group'. For crude mortality, I multiplied the age group populations by the age-specific mortality rates, the summed all the products before dividing by the total population. The Age specific mortality rate was a bit more straightforward in that I multiplied the age-specific rates by the WHO age standardization values, before adding them up. Because the age-specific mortality rates were already given per 100,000 people, I did not need to multiply by anything to produce deaths per 100,000.

### Given more time, I would have liked to produce a more efficient way to calculate the metrics for a given country, with a custom function with an argument of the country name.

In [4]:
import pandas as pd

In [18]:
COPD = pd.read_excel(r"age-specific death rates of COPD.xlsx")
fiveyrpop = pd.read_excel(r'WPP2022_POP_F02_1_POPULATION_5-YEAR_AGE_GROUPS_BOTH_SEXES.xlsx', sheet_name = 'Estimates', skiprows= 16)
WHO = pd.read_excel(r'WHO.xlsx', skiprows = 1)

In [53]:
COPD.head()

,Age group,"Death rate, United States, 2019","Death rate, Uganda, 2019"
0,0-4,0.04,0.40
1,5-9,0.02,0.17
2,10-14,0.02,0.07
3,15-19,0.02,0.23
4,20-24,0.06,0.38


In [52]:
COPD.rename(columns={'Age group (years)': 'Age group'}, inplace=True)

In [ ]:
# Combine the death rate columns into a single column
combined_df['Combined_Death_Rate'] = pd.concat([['death_rate_country1'], df2['death_rate_country2']], ignore_index=True)

# Drop the individual death rate columns if needed
combined_df.drop(columns=['death_rate_country1', 'death_rate_country2'], inplace=True)

## Wrangling the Five Year Population Estimates

In [17]:
fiveyrpop.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
0,1,Estimates,WORLD,NaN,900,NaN,NaN,1.0,World,0,...,85393.634,70796.6355,53122.984,36652.55,22440.709,10870.494,3977.119,969.3095,159.9325,14.469
1,2,Estimates,WORLD,NaN,900,NaN,NaN,1.0,World,0,...,87045.7935,71417.9795,53923.774,37034.4615,22701.1225,10913.2255,3958.666,966.8275,147.466,15.671
2,3,Estimates,WORLD,NaN,900,NaN,NaN,1.0,World,0,...,89000.4945,71930.168,54969.3695,37521.5325,22937.171,11036.6195,3956.967,966.5605,144.5635,16.1695
3,4,Estimates,WORLD,NaN,900,NaN,NaN,1.0,World,0,...,91230.957,72475.1835,56236.375,38144.896,23140.1195,11245.749,3973.4965,972.536,144.159,16.048
4,5,Estimates,WORLD,NaN,900,NaN,NaN,1.0,World,0,...,93357.6225,73358.0965,57461.302,38867.149,23379.9635,11452.133,4021.7115,982.418,145.427,15.426


In [ ]:
# Filter to 2019 and US and Uganda

In [37]:
fivefilter= fiveyrpop[(fiveyrpop['Year']==2019) & fiveyrpop['Region, subregion, country or area *'].isin(['United States of America', 'Uganda'])]

In [40]:
fivefilter = fivefilter.iloc[:, [2] + list(range(11, len(fivefilter.columns)))]
fivefilter

,"Region, subregion, country or area *",0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
2953,Uganda,7328.968,6614.4205,5899.4005,5151.082,4348.173,3499.504,2618.559,1903.175,1503.6695,...,687.315,500.2975,353.2215,197.1705,92.682,43.893,15.4255,3.5645,0.6165,0.096
18505,United States of America,19848.556,20697.0755,22092.1675,21895.1225,21871.8085,23406.9275,22842.1515,22296.9525,20694.5555,...,22347.5005,20941.0635,17500.8715,13688.5955,9272.809,6118.8665,3977.1775,1656.067,501.7545,78.955


In [ ]:
#combine last four age groups into 85+

In [45]:
last4 = fivefilter.iloc[:,-4:]
fivefilter = fivefilter.drop(fivefilter.columns[-4:], axis = 1)
fivefilter['85+'] = last4.sum(axis=1)

In [46]:
fivefilter

,"Region, subregion, country or area *",0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
2953,Uganda,7328.968,6614.4205,5899.4005,5151.082,4348.173,3499.504,2618.559,1903.175,1503.6695,1234.979,952.8675,687.315,500.2975,353.2215,197.1705,92.682,43.893,19.7025
18505,United States of America,19848.556,20697.0755,22092.1675,21895.1225,21871.8085,23406.9275,22842.1515,22296.9525,20694.5555,21244.2585,21346.4345,22347.5005,20941.0635,17500.8715,13688.5955,9272.809,6118.8665,6213.954


In [47]:
fivefilter['Total'] = fivefilter.iloc[:, 1:].sum(axis=1)
fivefilter

,"Region, subregion, country or area *",0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total
2953,Uganda,7328.968,6614.4205,5899.4005,5151.082,4348.173,3499.504,2618.559,1903.175,1503.6695,1234.979,952.8675,687.315,500.2975,353.2215,197.1705,92.682,43.893,19.7025,42949.08
18505,United States of America,19848.556,20697.0755,22092.1675,21895.1225,21871.8085,23406.9275,22842.1515,22296.9525,20694.5555,21244.2585,21346.4345,22347.5005,20941.0635,17500.8715,13688.5955,9272.809,6118.8665,6213.954,334319.6705


In [50]:
#melt dataframe 
melted_five = fivefilter.melt(id_vars=['Region, subregion, country or area *'], var_name='Age group', value_name='population')
melted_five

,"Region, subregion, country or area *",Age group,population
0,Uganda,0-4,7328.968
1,United States of America,0-4,19848.556
2,Uganda,5-9,6614.4205
3,United States of America,5-9,20697.0755
4,Uganda,10-14,5899.4005
5,United States of America,10-14,22092.1675
6,Uganda,15-19,5151.082
7,United States of America,15-19,21895.1225
8,Uganda,20-24,4348.173
9,United States of America,20-24,21871.8085


In [51]:
melted_five.rename(columns={'Region, subregion, country or area *': 'Country'}, inplace=True)
melted_five

,Country,Age group,population
0,Uganda,0-4,7328.968
1,United States of America,0-4,19848.556
2,Uganda,5-9,6614.4205
3,United States of America,5-9,20697.0755
4,Uganda,10-14,5899.4005
5,United States of America,10-14,22092.1675
6,Uganda,15-19,5151.082
7,United States of America,15-19,21895.1225
8,Uganda,20-24,4348.173
9,United States of America,20-24,21871.8085


In [58]:
UganAges = melted_five[melted_five['Country']=='Uganda']
USAges = melted_five[melted_five['Country']=='United States of America']

## Wrangling WHO Population proportions

In [19]:
WHO.head()

,Age group,"Segi ("" world"" ) standard","Scandinavian (""European"") standard",WHO World Standard*
0,0-4,12.0,8,8.86
1,5-9,10.0,7,8.69
2,10-14,9.0,7,8.60
3,15-19,9.0,7,8.47
4,20-24,8.0,7,8.22


In [43]:
WHO = WHO.drop(WHO.columns[[1, 2]], axis=1)
WHO

,Age group,WHO World Standard*
0,0-4,8.86
1,5-9,8.69
2,10-14,8.60
3,15-19,8.47
4,20-24,8.22
5,25-29,7.93
6,30-34,7.61
7,35 39,7.15
8,40-44,6.59
9,45-49,6.04


## Crude and Age Specific Mortality Rate

In [99]:
#Uganda crude mortality
merged_data = pd.merge(UganAges, COPD, on='Age group')
merged_data = pd.merge(merged_data, WHO, on='Age group')
# Calculate mortality count by age group
merged_data['Ugan_mortality_count'] = merged_data['population'] * merged_data['Death rate, Uganda, 2019'] 
merged_data['weighted_mortality'] = merged_data['Death rate, Uganda, 2019'] * merged_data['WHO World Standard*'] /100
# Calculate mortality rate per 100,000 population
crude_mortality_rate = merged_data['Ugan_mortality_count'].sum() / UganAges.loc[UganAges['Age group'] == 'Total', 'population']
age_standardized_mortality_rate = merged_data['weighted_mortality'].sum()

# Display the resulting crude mortality rate
print('Uganda crude mortality rate: ' + str(round(float(crude_mortality_rate), 1)))
print('Uganda Age Standardized mortality rate: ' + str(round(float(age_standardized_mortality_rate), 1)))

Uganda crude mortality rate: 5.8
Uganda Age Standardized mortality rate: 28.6


/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52605/1928022164.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print('Uganda crude mortality rate: ' + str(round(float(crude_mortality_rate), 1)))


In [97]:
#US crude mortality
merged_data = pd.merge(USAges, COPD, on='Age group')
merged_data = pd.merge(merged_data, WHO, on='Age group')
# Calculate mortality count by age group
merged_data['US_mortality_count'] = merged_data['population'] * merged_data['Death rate, United States, 2019']
merged_data['weighted_mortality'] = merged_data['Death rate, United States, 2019'] * merged_data['WHO World Standard*'] / 100
# Calculate mortality rate per 100,000 population
crude_mortality_rate = merged_data['US_mortality_count'].sum() / USAges.loc[USAges['Age group'] == 'Total', 'population']
age_standardized_mortality_rate = merged_data['weighted_mortality'].sum()

# Display the resulting crude mortality rate
print('US crude mortality rate: ' + str(round(float(crude_mortality_rate), 1)))
print('US Age Standardized mortality rate: ' + str(round(float(age_standardized_mortality_rate), 1)))

US crude mortality rate: 57.2
US Age Standardized mortality rate: 28.4


/var/folders/bs/zggx74yj4d721rnp57yp6h5r0000gn/T/ipykernel_52605/2621505918.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print('US crude mortality rate: ' + str(round(float(crude_mortality_rate), 1)))
